### Cost prediction using linear regression algorithm

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
t=pd.read_csv("C:\\Users\\intel\\Downloads\\Manpower_SF_Nov12.csv")

In [3]:
t.drop(columns=["plan_cd","dept","division", "direct_indirect",
                 "employee_class","workforce_category_descr","attend_reason_code",
                 "plant_line_running"],inplace=True)

In [4]:
t.rename(columns={'plant_code':'classes','absentee':'cost','total_attend_hours':'hours',
                  'jobtitle':'job','attend_type_code':'code'},inplace=True)

In [5]:
start_date = ('2023-10-01')
end_date =('2023-11-30')
t1=t[(t['attend_date'] >= start_date) & (t['attend_date'] <= end_date)]
t1

,Unnamed: 0,classes,attend_date,job,code,cost,hours
1,1,AEP,2023-11-10,LSS,ABS,1,8.0
18,19,AAP,2023-10-05,PA,PTO,1,4.0
42,45,AEP,2023-10-25,PA,ABS,1,8.0
54,57,AAP,2023-10-31,PA,ABS,5,40.0
59,62,MAP,2023-10-23,PA,ABS,9,72.0
...,...,...,...,...,...,...,...
396799,22274,AEP,2023-10-06,PMA,PTO,1,8.0
396804,22279,MAP,2023-10-24,LSS,ABS,1,8.0
396848,22330,AEP,2023-10-12,PA,PTO,2,16.0
396853,22336,TPO,2023-10-13,PAN,ABS,1,8.0


In [6]:
t1['attend_date'] = pd.to_datetime(t1['attend_date'])

# Extract year, month, and day into separate columns
t1['year'] = t1['attend_date'].dt.year
t1['month'] = t1['attend_date'].dt.month
t1['day'] = t1['attend_date'].dt.day

# Display the resulting DataFrame
print(t1[['attend_date', 'year', 'month', 'day']])

       attend_date  year  month  day
1       2023-11-10  2023     11   10
18      2023-10-05  2023     10    5
42      2023-10-25  2023     10   25
54      2023-10-31  2023     10   31
59      2023-10-23  2023     10   23
...            ...   ...    ...  ...
396799  2023-10-06  2023     10    6
396804  2023-10-24  2023     10   24
396848  2023-10-12  2023     10   12
396853  2023-10-13  2023     10   13
396860  2023-10-06  2023     10    6

[19596 rows x 4 columns]


In [7]:
t1

,Unnamed: 0,classes,attend_date,job,code,cost,hours,year,month,day
1,1,AEP,2023-11-10,LSS,ABS,1,8.0,2023,11,10
18,19,AAP,2023-10-05,PA,PTO,1,4.0,2023,10,5
42,45,AEP,2023-10-25,PA,ABS,1,8.0,2023,10,25
54,57,AAP,2023-10-31,PA,ABS,5,40.0,2023,10,31
59,62,MAP,2023-10-23,PA,ABS,9,72.0,2023,10,23
...,...,...,...,...,...,...,...,...,...,...
396799,22274,AEP,2023-10-06,PMA,PTO,1,8.0,2023,10,6
396804,22279,MAP,2023-10-24,LSS,ABS,1,8.0,2023,10,24
396848,22330,AEP,2023-10-12,PA,PTO,2,16.0,2023,10,12
396853,22336,TPO,2023-10-13,PAN,ABS,1,8.0,2023,10,13


In [8]:
from sklearn import preprocessing                    #import label encoder
label_encoder= preprocessing.LabelEncoder()          #label_encoder object knows how to understand word labels
t1['classes'] = label_encoder.fit_transform(t1['classes'])     
t1['classes'].value_counts()                           

classes
0    5070
1    4398
4    3754
3    2754
2    2414
6     792
5     414
Name: count, dtype: int64

In [9]:
t1['job'] = label_encoder.fit_transform(t1['job'])     
t1['job'].value_counts() 

job
2    14615
0     2303
5     1253
3      505
4      456
1      322
6      142
Name: count, dtype: int64

In [10]:
t1['code'] = label_encoder.fit_transform(t1['code'])    
t1['code'].value_counts

<bound method IndexOpsMixin.value_counts of 1         0
18        3
42        0
54        0
59        0
         ..
396799    3
396804    0
396848    3
396853    0
396860    0
Name: code, Length: 19596, dtype: int32>

In [11]:
t1

,Unnamed: 0,classes,attend_date,job,code,cost,hours,year,month,day
1,1,1,2023-11-10,0,0,1,8.0,2023,11,10
18,19,0,2023-10-05,2,3,1,4.0,2023,10,5
42,45,1,2023-10-25,2,0,1,8.0,2023,10,25
54,57,0,2023-10-31,2,0,5,40.0,2023,10,31
59,62,4,2023-10-23,2,0,9,72.0,2023,10,23
...,...,...,...,...,...,...,...,...,...,...
396799,22274,1,2023-10-06,4,3,1,8.0,2023,10,6
396804,22279,4,2023-10-24,0,0,1,8.0,2023,10,24
396848,22330,1,2023-10-12,2,3,2,16.0,2023,10,12
396853,22336,6,2023-10-13,3,0,1,8.0,2023,10,13


In [12]:
x=t1[["classes",'job','code','hours','year','month','day']] 
y=t1['cost'].values  

In [13]:
#spliting the data
from sklearn.model_selection import train_test_split
x_train ,x_test, y_train, y_test = train_test_split (x,y,test_size=0.3,random_state=0) 

In [14]:
#Create a linear regression model 
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 

In [15]:
#fit the model
regressor.fit(x_train, y_train) 

LinearRegression()

In [16]:
lrpred = regressor.predict(x_test)
lrpred

array([1.01909952, 3.9749428 , 1.0240398 , ..., 0.52513876, 2.12544709,
       0.62134397])

In [17]:
regressor.score(x_test,y_test)

0.9984403204793859

In [18]:
from sklearn.metrics import mean_squared_error 
mse = mean_squared_error(y_test, lrpred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.05109294626848197


### Hours prediction by using linear regression  algorithm

In [28]:
t1

,Unnamed: 0,classes,attend_date,job,code,cost,hours,year,month,day
1,1,1,2023-11-10,0,0,1,8.0,2023,11,10
18,19,0,2023-10-05,2,3,1,4.0,2023,10,5
42,45,1,2023-10-25,2,0,1,8.0,2023,10,25
54,57,0,2023-10-31,2,0,5,40.0,2023,10,31
59,62,4,2023-10-23,2,0,9,72.0,2023,10,23
...,...,...,...,...,...,...,...,...,...,...
396799,22274,1,2023-10-06,4,3,1,8.0,2023,10,6
396804,22279,4,2023-10-24,0,0,1,8.0,2023,10,24
396848,22330,1,2023-10-12,2,3,2,16.0,2023,10,12
396853,22336,6,2023-10-13,3,0,1,8.0,2023,10,13


In [29]:
A=t1[["classes",'job','code','cost','year','month','day']] 
B=t1['hours'].values  

In [30]:
from sklearn.model_selection import train_test_split
x1_train ,x1_test, y1_train, y1_test = train_test_split (A,B,test_size=0.2,random_state=0) 

In [31]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 
regressor.fit(x1_train, y1_train)

LinearRegression()

In [32]:
lrpred = regressor.predict(x1_test)
lrpred

array([ 7.86509466, 32.20575966,  7.84236678, ..., 15.03732915,
       15.88007356,  7.01366678])

In [33]:
regressor.score(x1_test,y1_test)

0.9985440467106749

In [34]:
from sklearn.metrics import mean_squared_error 
mse = mean_squared_error(y1_test, lrpred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 3.3506434666134504


In [35]:
result = pd.DataFrame({'Actual cost': y1_test, 'Predicted cost': lrpred})
print(result)

      Actual cost  Predicted cost
0             8.0        7.865095
1            32.0       32.205760
2             8.0        7.842367
3            16.0       15.834536
4            88.0       87.892562
...           ...             ...
3915        104.0      103.863640
3916          8.0        7.875178
3917         16.0       15.037329
3918          8.0       15.880074
3919          4.0        7.013667

[3920 rows x 2 columns]
